In [1]:
#import lightgbm
import random
import pandas as pd
import numpy as np
import os
import time
t = time.strftime('%m%d-%H%M', time.localtime(time.time()))
import warnings
warnings.filterwarnings(action='ignore')

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정
#데이터 불러오기
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
building_info =  pd.read_csv('./data/building_info.csv')

train_df = train_df.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
test_df = test_df.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
building_info = building_info.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})
translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '대학교': 'University',
    '데이터센터': 'Data Center',
    '백화점및아울렛': 'Department Store and Outlet',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    '지식산업센터': 'Knowledge Industry Center',
    '할인마트': 'Discount Mart',
    '호텔및리조트': 'Hotel and Resort'
}

building_info['building_type'] = building_info['building_type'].replace(translation_dict)


train_df.head()

,num_date_time,building_number,date_time,temperature,rainfall,windspeed,humidity,sunshine,solar_radiation,power_consumption
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


In [2]:
pd.set_option('display.max_columns', 30)

def merge(df):
    df = df.fillna(0)
    date = pd.to_datetime(df.date_time)
    df['hour'] = date.dt.hour
    df['day'] = date.dt.weekday
    df['month'] = date.dt.month
    df['week'] = date.dt.isocalendar().week
    
    # 'building_number'를 기준으로 두 데이터프레임 병합 및 전처리
    merged_df = pd.merge(df, building_info, on='building_number',how='right')
    #merge후 전처리
    merged_df['solar_power_capacity'] = merged_df['solar_power_capacity'].replace('-', 0)
    merged_df['ess_capacity'] = merged_df['ess_capacity'].replace('-', 0)
    merged_df['pcs_capacity'] = merged_df['pcs_capacity'].replace('-', 0)

    merged_df['solar_power_capacity'] = merged_df['solar_power_capacity'].astype('float64')
    merged_df['ess_capacity'] = merged_df['ess_capacity'].astype('float64')
    merged_df['pcs_capacity'] = merged_df['pcs_capacity'].astype('float64')
    return merged_df

merged_train_df0 = merge(train_df)
merged_test_df0 = merge(test_df)


kk = merged_train_df0.loc[(merged_train_df0.building_number == 95)&(merged_train_df0.hour == 16)&(merged_train_df0.day == 2), 'power_consumption']
sum16 = kk.sum()/12
kkk = merged_train_df0.loc[(merged_train_df0.building_number == 95)&(merged_train_df0.hour == 17)&(merged_train_df0.day == 2), 'power_consumption']
sum17 = (kkk.sum()-0.36)/12
merged_train_df0.loc[(merged_train_df0.building_number == 95)&(merged_train_df0.hour == 16)&(merged_train_df0.day == 2)&(merged_train_df0.month == 7)&(merged_train_df0.week == 30), 'power_consumption'] = sum16
merged_train_df0.loc[(merged_train_df0.building_number == 95)&(merged_train_df0.hour == 17)&(merged_train_df0.day == 2)&(merged_train_df0.month == 7)&(merged_train_df0.week == 30), 'power_consumption'] = sum17

In [3]:
def preprocessing(df):
    df['holiday'] = df.apply(lambda x: 0 if x['day'] < 5 else 1, axis=1)
    df.loc[((df['month'] == 6) & (df['day'] == 2) &(df['week'] == 22), 'holiday')] = 1
    df.loc[((df['month'] == 6) & (df['day'] == 0) &(df['week'] == 23), 'holiday')] = 1
    df.loc[((df['month'] == 8) & (df['day'] == 0) &(df['week'] == 33), 'holiday')] = 1
    #######################################
    ## 건물별, 요일별, 전력시간별 소비량 평균 넣어주기
    #######################################
    power_mean = pd.pivot_table(merged_train_df0, values = 'power_consumption', index = ['building_number', 'hour', 'day'], aggfunc = np.mean).reset_index()
    #######################################
    ## 건물별, 요일별, 전력시간별 소비량 표준편차 넣어주기
    #######################################
    power_std = pd.pivot_table(merged_train_df0, values = 'power_consumption', index = ['building_number', 'hour', 'day'], aggfunc = np.std).reset_index()
    #######################################
    ## 건물별 시간별 전력소비량 평균 넣어주기
    #######################################
    power_hour_mean = pd.pivot_table(merged_train_df0, values = 'power_consumption', index = ['building_number', 'hour'], aggfunc = np.mean).reset_index()
    #######################################
    ## 건물별 시간별 전력소비량 표준편차 넣어주기
    #######################################
    power_hour_std = pd.pivot_table(merged_train_df0, values = 'power_consumption', index = ['building_number', 'hour'], aggfunc = np.std).reset_index()
    
    # 병합을 위한 키 설정
    merge_keys = ['building_number', 'hour', 'day']  
    # 데이터프레임 병합
    df = df.merge(power_mean[merge_keys + ['power_consumption']], on=merge_keys, how='left', suffixes=('', '_day_hour_mean'))
    df = df.merge(power_std[merge_keys + ['power_consumption']], on=merge_keys, how='left', suffixes=('', '_day_hour_std'))
    df = df.merge(power_hour_mean[merge_keys[:-1] + ['power_consumption']], on=merge_keys[:-1], how='left', suffixes=('', '_hour_mean'))
    df = df.merge(power_hour_std[merge_keys[:-1] + ['power_consumption']], on=merge_keys[:-1], how='left', suffixes=('', '_hour_std'))
    df = df.rename(columns = {'power_consumption_day_hour_mean':'day_hour_mean','power_consumption_day_hour_std':'day_hour_std','power_consumption_hour_mean':'hour_mean','power_consumption_hour_std':'hour_std',})
    
    def CDH(xs):
        ys = []
        for i in range(len(xs)):
            if i < 11:
                ys.append(np.sum(xs[:(i+1)]-26))
            else:
                ys.append(np.sum(xs[(i-11):(i+1)]-26))
        return np.array(ys)

    cdhs = np.array([])
    for num in range(1,101,1):
        temp = df[df['building_number'] == num]
        cdh = CDH(temp['temperature'].values)
        cdhs = np.concatenate([cdhs, cdh])
    df['CDH'] = cdhs
     
    ## https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
    df['sin_time'] = np.sin(2*np.pi*df.hour/24)
    df['cos_time'] = np.cos(2*np.pi*df.hour/24)
    
    ## https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent
    df['THI'] = 9/5*df['temperature'] - 0.55*(1-df['humidity']/100)*(9/5*df['humidity']-26)+32
    
    return df

merged_train_df1 = preprocessing(merged_train_df0)
merged_test_df1 = preprocessing(merged_test_df0).rename(columns = {'power_consumption':'day_hour_mean'})

In [4]:
train = pd.DataFrame()
vaild = pd.DataFrame()
for i in range(1,101,1):
    buff = merged_train_df1.loc[(merged_train_df1.building_number == i)] 
    train = pd.concat([train, buff[:-168]])
    vaild = pd.concat([vaild, buff[-168:]])

In [5]:
def data_train(df):
    
    grouped = df.groupby(['building_number', 'date_time'])
    df['max_power'] = grouped['power_consumption'].transform(np.max)
    df['min_power'] = grouped['power_consumption'].transform(np.min)
    
    grouped2 = df.groupby(['building_number', 'day'])
    df['max_power'] = grouped2['max_power'].transform(np.mean)
    df['min_power'] = grouped2['min_power'].transform(np.mean)

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0,inplace=True)
    
    return df.drop(columns=['num_date_time','date_time','building_type','sunshine','pcs_capacity','ess_capacity','solar_power_capacity','solar_radiation','total_area','cooling_area'])

def data_test(df):
    grouped_train = merged_train_df.groupby(['building_number', 'day'])
    max_power = grouped_train['max_power'].median()
    min_power = grouped_train['min_power'].median()
    max_power_df = max_power.reset_index()
    min_power_df = min_power.reset_index()

    # 테스트 데이터에 max_power_median와 min_power_median을 추가
    df = pd.merge(df, max_power_df, on=['building_number', 'day'])
    df = pd.merge(df, min_power_df, on=['building_number', 'day'])
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0,inplace=True)

    return df.drop(columns=['num_date_time','date_time','building_type','pcs_capacity','ess_capacity','solar_power_capacity','total_area','cooling_area'])

merged_train_df =data_train(merged_train_df1)
merged_test_df =data_test(merged_test_df1)

In [7]:
train = data_train(train)
vaild = data_test(vaild).drop(columns=['sunshine','solar_radiation'])

train, test 저장

In [8]:
## save the preprocessed data
merged_train_df.to_csv(f'./split/train_preprocessed_wsw_{t}.csv')
merged_test_df.to_csv(f'./split/test_preprocessed_wsw_{t}.csv')

vaild저장

In [9]:
## save the preprocessed data
train.to_csv(f'./split/train_split_preprocessed_wsw_{t}.csv')
vaild.to_csv(f'./split/vaild_preprocessed_wsw_{t}.csv')

빌딩별로 csv생성

In [28]:
#train,test
train_df  = pd.read_csv((f'./split/train_preprocessed_wsw_0803-0021.csv')).drop(columns='Unnamed: 0')
test_df  = pd.read_csv((f'./split/vaild_preprocessed_wsw_0803-0021.csv')).drop(columns='Unnamed: 0')
for i in range(1,101,1):
    tr = train_df.loc[train_df.building_number==i].reset_index()
    tr = tr.drop(columns=['index'])
    te = test_df.loc[test_df.building_number==i].reset_index()
    te = te.drop(columns=['index'])
    tr.to_csv(f'./split/train_building{i}.csv')
    te.to_csv(f'./split/test_building{i}.csv')

In [29]:
#train,valid
train  = pd.read_csv((f'./split/train_split_preprocessed_wsw_0803-0021.csv')).drop(columns='Unnamed: 0')
vaild  = pd.read_csv((f'./split/vaild_preprocessed_wsw_0803-0021.csv')).drop(columns='Unnamed: 0')
for i in range(1,101,1):
    trv = train.loc[train.building_number==i].reset_index()
    trv = trv.drop(columns=['index'])
    v = vaild.loc[vaild.building_number==i].reset_index()
    v = v.drop(columns=['index'])
    trv.to_csv(f'./split/train_valid_building{i}.csv')
    v.to_csv(f'./split/valid_building{i}.csv')

KeyError: "['index'] not found in axis"